In [63]:
from torch.utils.data import Subset
import numpy as np
from torch import nn, optim
from torchvision import transforms
import torch
import torchvision
import sklearn.model_selection as sc

transform = transforms.Compose([transforms.ToTensor()])
data = torchvision.datasets.FashionMNIST('path', download=True,transform=transform)

data_loader = torch.utils.data.DataLoader(data, batch_size=16, shuffle=True)
from torch.utils.data import random_split, DataLoader


total_size = len(data)
train_size = int(0.3 * total_size)  
test_size = total_size - train_size 

train_dataset, test_dataset= random_split(data, [train_size, test_size])
num_samples_to_modify = int(0.1 * len(train_dataset))
num_samples_to_modify_test = int(0.1 * len(test_dataset))

subset_indices = np.random.choice(len(train_dataset), num_samples_to_modify, replace=False)
subset_indices_test = np.random.choice(len(test_dataset), num_samples_to_modify_test, replace=False)
mean = 0
std_dev = 0.1  

for index in subset_indices:
    image, label = train_dataset[index]
    noise = np.random.normal(mean, std_dev, image.shape)
    noisy_image = image.numpy() + noise
    train_dataset.dataset.data[index] = torch.from_numpy(np.clip(noisy_image, 0, 1))

for index in subset_indices_test:
    image, label = test_dataset[index]
    noise = np.random.normal(mean, std_dev, image.shape)
    noisy_image = image.numpy() + noise
    test_dataset.dataset.data[index] = torch.from_numpy(np.clip(noisy_image, 0, 1))  

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

print( train_size, test_size)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2,padding=0)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2,padding=0)

        self.fc1 = nn.LazyLinear(64 * 7 * 7)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.LazyLinear(100)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')



18000 42000
Epoch [1/5], Step [100/1125], Loss: 0.9741197824478149
Epoch [1/5], Step [200/1125], Loss: 0.43621763586997986
Epoch [1/5], Step [300/1125], Loss: 0.7011340856552124
Epoch [1/5], Step [400/1125], Loss: 0.4736374616622925
Epoch [1/5], Step [500/1125], Loss: 0.59006267786026
Epoch [1/5], Step [600/1125], Loss: 0.3246796727180481
Epoch [1/5], Step [700/1125], Loss: 0.8025538921356201
Epoch [1/5], Step [800/1125], Loss: 0.7618135213851929
Epoch [1/5], Step [900/1125], Loss: 0.38286104798316956
Epoch [1/5], Step [1000/1125], Loss: 0.6809604167938232
Epoch [1/5], Step [1100/1125], Loss: 0.24529492855072021
Epoch [2/5], Step [100/1125], Loss: 0.42182061076164246
Epoch [2/5], Step [200/1125], Loss: 0.44835638999938965
Epoch [2/5], Step [300/1125], Loss: 0.5831490755081177
Epoch [2/5], Step [400/1125], Loss: 0.8792046308517456
Epoch [2/5], Step [500/1125], Loss: 0.8680644035339355
Epoch [2/5], Step [600/1125], Loss: 0.5299698114395142
Epoch [2/5], Step [700/1125], Loss: 0.3984332680